In [1]:
import spotipy
from youtube_search import YoutubeSearch
import concurrent.futures as cf
import yt_dlp

client_id = "CLIENT_ID"
client_secret = "CLIENT_SECRET"
redirect_uri = "http://localhost:3000/dashboard"
scope = "user-library-read"
username = "mistercouscous"

class DownloadMusic:
    def __init__(self, client_id: str, client_secret: str, redirect_uri: str, scope: str, username: str):
        self.client_id = client_id
        self.client_secret = client_secret
        self.redirect_uri = redirect_uri
        self.scope = scope
        self.username = username
        self.sp = spotipy.Spotify(
            auth_manager=spotipy.SpotifyClientCredentials(
                client_id=client_id, client_secret=client_secret
            )
        )
        self.playlistName = None

    def write_playlist(self, username: str, playlist_id: str):
        results = self.sp.user_playlist(username, playlist_id)
        playlist_name = results['name']
        self.playlistName = playlist_name
        text_file = f"{playlist_name}.txt"
        print(f"Writing {results['tracks']['total']} tracks to {text_file}.")
        tracks = results['tracks']
        self.write_tracks(text_file, tracks)
        return playlist_name

    def write_tracks(self, text_file: str, tracks: dict):
        with open(text_file, 'w+', encoding='utf-8') as file_out:
            while True:
                for item in tracks['items']:
                    if 'track' in item:
                        track = item['track']
                    else:
                        track = item
                    try:
                        track_url = track['external_urls']['spotify']
                        track_name = track['name']
                        track_artist = track['artists'][0]['name']
                        csv_line = f"{track_name},{track_artist},{track_url}\n"
                        try:
                            file_out.write(csv_line)
                        except UnicodeEncodeError:
                            print(f"Track named {track_name} failed due to a UnicodeEncodeError. This is most likely due to the track being unavailable in your region.")
                    except KeyError:
                        print(f"Track named {track['name']} failed due to a KeyError. This is most likely due to the track being unavailable in your region.")
                if tracks['next']:
                    tracks = self.sp.next(tracks)
                else:
                    break

    def find_and_download_songs_spotify(self, reference_file: str):
        TOTAL_ATTEMPTS = 10
        with open(reference_file, "r", encoding='utf-8') as file:
            for line in file:
                temp = line.split(",")
                name, artist = temp[0], temp[1]
                print(f"Searching for {name} by {artist}.")
                text_to_search = f"{name}-{artist}"
                best_url = None
                attempts_left = TOTAL_ATTEMPTS
                while attempts_left > 0:
                    try:
                        results_list = YoutubeSearch(text_to_search, max_results=1).to_dict()
                        best_url = f"https://www.youtube.com{results_list[0]['url_suffix']}"
                        break
                    except IndexError:
                        attempts_left -= 1
                        print(f"No valid URLs found for {text_to_search}, trying again ({attempts_left} attempts left).")
                if best_url is None:
                    print(f"No valid URLs found for {text_to_search}, skipping track.")
                    continue
                print(f"Initiating download for {text_to_search}.")
                ydl_opts = {
                    'format': 'bestaudio/best',
                    'postprocessors': [{
                        'key': 'FFmpegExtractAudio',
                        'preferredcodec': 'wav',
                        'preferredquality': '320',
                    }],
                    'outtmpl': f'{self.playlistName}/%(title)s.%(ext)s',
                    'ffmpeg_location': '/usr/bin/ffmpeg',
                }
                with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                    ydl.download([best_url])

    def run(self, playlist_id: str):
        self.sp.playlist_tracks(playlist_id)
        playlist_uri = f"spotify:playlist:{playlist_id}"
        playlist_name = self.write_playlist(self.username, playlist_uri)
        reference_file = f"{playlist_name}.txt"
        self.find_and_download_songs_spotify(reference_file)

playlist_ids = ["37i9dQZF1DX6J5NfMJS675", "3B7HTAQWys3xZZ8YChBgUE", "2rs8eQ5qDqu2lLIqHvId6l"]
if __name__ == "__main__":
    with cf.ThreadPoolExecutor(max_workers=len(playlist_ids)) as executor:
        for playlist_id in playlist_ids:
            executor.submit(DownloadMusic(client_id, client_secret, redirect_uri, scope, username).run, playlist_id)


Writing 75 tracks to TECHNO BUNKER.txt.
Searching for Hare Ram by Lilly Palmer.
Writing 85 tracks to SV24 - Hard.txt.
Searching for MASSACRO by Malice.
Writing 331 tracks to DLDS-reggaeton.txt.
Initiating download for MASSACRO-Malice.
[youtube] Extracting URL: https://www.youtube.com/watch?v=pRj8shjopKM&pp=ygUPTUFTU0FDUk8tTWFsaWNl
[youtube] pRj8shjopKM: Downloading webpage
Initiating download for Hare Ram-Lilly Palmer.
[youtube] Extracting URL: https://www.youtube.com/watch?v=JKivvqaOyd0&pp=ygUVSGFyZSBSYW0tTGlsbHkgUGFsbWVy
[youtube] JKivvqaOyd0: Downloading webpage
[youtube] pRj8shjopKM: Downloading ios player API JSON
[youtube] pRj8shjopKM: Downloading android player API JSON


[youtube] JKivvqaOyd0: Downloading ios player API JSON
[youtube] JKivvqaOyd0: Downloading android player API JSON


Searching for NANA DEL HILO ROJO by SAIKO.
Initiating download for NANA DEL HILO ROJO-SAIKO.
[youtube] Extracting URL: https://www.youtube.com/watch?v=RfMiJGkEYW8&pp=ygUYTkFOQSBERUwgSElMTyBST0pPLVNBSUtP
[youtube] RfMiJGkEYW8: Downloading webpage
[youtube] pRj8shjopKM: Downloading m3u8 information


[youtube] pRj8shjopKM: Downloading initial data API JSON
[youtube] RfMiJGkEYW8: Downloading ios player API JSON
[youtube] RfMiJGkEYW8: Downloading android player API JSON


[youtube] JKivvqaOyd0: Downloading m3u8 information
[info] pRj8shjopKM: Downloading 1 format(s): 251
[info] JKivvqaOyd0: Downloading 1 format(s): 251
[download] Destination: SV24 - Hard/Aggressive Act - MASSACRO (Official Video) (AR Mixtape Vol.1).webm
[download]   3.3% of    1.88MiB at  329.72KiB/s ETA 00:05[download] Destination: TECHNO BUNKER/Lilly Palmer - Hare Ram (Official Music Video).webm
[download] 100% of    1.88MiB in 00:00:01 at 1.61MiB/s   
[ExtractAudio] Destination: SV24 - Hard/Aggressive Act - MASSACRO (Official Video) (AR Mixtape Vol.1).wav
[download] 100.0% of    3.36MiB at    2.18MiB/s ETA 00:00Deleting original file SV24 - Hard/Aggressive Act - MASSACRO (Official Video) (AR Mixtape Vol.1).webm (pass -k to keep)
[download] 100% of    3.36MiB in 00:00:01 at 1.97MiB/s   
Searching for Pussy Motherfuckerz - D-Fence Remix by Endymion.
[ExtractAudio] Destination: TECHNO BUNKER/Lilly Palmer - Hare Ram (Official Music Video).wav
[youtube] RfMiJGkEYW8: Downloading m3u8 infor

...

KeyboardInterrupt: 

Deleting original file TECHNO BUNKER/Space 92 & HI-LO - ORION [Official Audio].webm (pass -k to keep)
Searching for When I Push - Edit by Eli Brown.
[youtube] mMifb5E2HJs: Downloading ios player API JSON
[youtube] mMifb5E2HJs: Downloading android player API JSON


Initiating download for When I Push - Edit-Eli Brown.
[youtube] Extracting URL: https://www.youtube.com/watch?v=hESkviT7kBQ&pp=ygUcV2hlbiBJIFB1c2ggLSBFZGl0LUVsaSBCcm93bg%3D%3D
[youtube] hESkviT7kBQ: Downloading webpage
[youtube] hESkviT7kBQ: Downloading ios player API JSON
[youtube] hESkviT7kBQ: Downloading android player API JSON


[youtube] mMifb5E2HJs: Downloading m3u8 information
[info] mMifb5E2HJs: Downloading 1 format(s): 251
[download] Destination: SV24 - Hard/The Prophet - Wanna Play？ (270 BPM Terror Edit) (Official Audio).webm
[download] 100% of    1.90MiB in 00:00:01 at 1.82MiB/s   
[ExtractAudio] Destination: SV24 - Hard/The Prophet - Wanna Play？ (270 BPM Terror Edit) (Official Audio).wav
Deleting original file SV24 - Hard/The Prophet - Wanna Play？ (270 BPM Terror Edit) (Official Audio).webm (pass -k to keep)
Searching for 50 Pence Genc - Radio Edit by Rami Ali.
[youtube] hESkviT7kBQ: Downloading m3u8 information
[info] hESkviT7kBQ: Downloading 1 format(s): 251
Initiating download for 50 Pence Genc - Radio Edit-Rami Ali.
[download] Destination: TECHNO BUNKER/Eli Brown, Layton Giordani & OFFAIAH - When I Push ｜ Drumcode.webm
[youtube] Extracting URL: https://www.youtube.com/watch?v=xIiGAQjBR-w&pp=ygUjNTAgUGVuY2UgR2VuYyAtIFJhZGlvIEVkaXQtUmFtaSBBbGk%3D
[download]   0.0% of    5.29MiB at   21.40KiB/s ETA 04

[download] 100% of    5.29MiB in 00:00:01 at 3.37MiB/s   
[ExtractAudio] Destination: TECHNO BUNKER/Eli Brown, Layton Giordani & OFFAIAH - When I Push ｜ Drumcode.wav
Deleting original file TECHNO BUNKER/Eli Brown, Layton Giordani & OFFAIAH - When I Push ｜ Drumcode.webm (pass -k to keep)
Searching for Breathe by Amelie Lens.
[youtube] xIiGAQjBR-w: Downloading m3u8 information
Initiating download for Breathe-Amelie Lens.
[youtube] Extracting URL: https://www.youtube.com/watch?v=FVCEeAf6Xag&pp=ygUTQnJlYXRoZS1BbWVsaWUgTGVucw%3D%3D
[youtube] FVCEeAf6Xag: Downloading webpage
[info] xIiGAQjBR-w: Downloading 1 format(s): 251
[download] Destination: SV24 - Hard/50 Pence Genc (Radio Edit).webm
[download]  27.1% of    3.68MiB at    4.04MiB/s ETA 00:00